In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import numpy as np

In [2]:
#Launch get_pages(int) to launch the program

In [3]:
#Lists which will store our datas during the script work, initialize them before launch program
pages=[]
emp=[]
price=[]
areas=[]
type_h=[]
description=[]
pieces_nb=[]
urls=[]
code = []
dep=[]

In [4]:
#Allows us to split zip_code from the emplacement
def zip_code(x):
    m=[]
    n=[]
    for c in x:
        m.append(re.findall('\s\d{1,2}|\(\d{2}\)|\(\d+\)',c))
     
    
    for c in m:
        n.append(str(''.join(c)))
   

    for s in n:
        code.append(s.replace('(','').replace(')','').replace(' ',''))
        

        
    for i in range(len(code)):
        if code[i] == '1':
            code[i] = '75001'
        if code[i] == '2':
            code[i] = '75002'
        if code[i] == '3':
            code[i] = '75003'
        if code[i] == '4':
            code[i] = '75004'
        if code[i] == '5':
            code[i] = '75005'
        if code[i] == '6':
            code[i] = '75006'
        if code[i] == '7':
            code[i] = '75007'
        if code[i] == '8':
            code[i] = '75008'
        if code[i] == '9':
            code[i] = '75009'
        if code[i] == '10':
            code[i] = '75010'
        if code[i] == '11':
            code[i] = '75011'
        if code[i] == '12':
            code[i] = '75012'
        if code[i] == '13':
            code[i] = '75013'
        if code[i] == '14':
            code[i] = '75014'
        if code[i] == '15':
            code[i] = '75015'
        if code[i] == '16':
            code[i] = '75016'
        if code[i] == '17':
            code[i] = '75017'
        if code[i] == '18':
            code[i] = '75018'
        if code[i] == '19':
            code[i] = '75019'
        if code[i] == '20':
            code[i] = '75020'

            
#Create the departements
def dep_(x):
    z = []
    a = []
    for i in x:
        z.append(re.findall('^\d{2}',i))
    for i in z:
        dep.append(str(''.join(i)))

In [5]:
def na_to_np(x,y) :
    x[y] = x[y].replace('NaN',np.NaN)
    

In [6]:
#Script that will convert our lists into dataframe and csv

def lists_to_df(a,b,c,d,e,f,g,h):
    names = ['Type', 'Nombre de Pièces', 'Surface (m²)','Prix (€)','Departement','Code Postal','Emplacement', 'Description']
    immo = pd.DataFrame(columns = names)
    
    immo['Type'] = a
    immo['Nombre de Pièces'] = b
    immo['Surface (m²)'] = c
    immo['Prix (€)'] = d
    immo['Departement'] = e
    immo['Code Postal'] = f
    immo['Emplacement'] = g
    immo['Description'] = h
    
    na_to_np(immo,'Nombre de Pièces')
    na_to_np(immo,'Code Postal')
    na_to_np(immo,'Surface (m²)')
    na_to_np(immo,'Departement')
    na_to_np(immo,'Type')
    na_to_np(immo,'Description')
    na_to_np(immo,'Emplacement')
    na_to_np(immo,'Prix (€)')
    
    immo = immo.dropna()
    immo = immo.drop_duplicates()
    immo = immo.fillna(0)
    
    immo.to_csv('ParuVendu.csv', index = False, header = True)
    

In [7]:
#Parse and clean data
def parse_pages(x, soup) :
    #This variable allows us to fill the unreachables values and have lists of same length to convert them into dataframe later
    bot = 'NaN'
    
    
    #This variables will be used in loops to help us select data we need
    prices = soup.find_all('div', class_='ergov3-priceannonce')
    title = soup.find_all('div', class_="ergov3-h3")
    desc = soup.find_all('p',class_='txt-long')
    
    #Box where we store all type of houses available
    model_title = 'Appartement|Maison|Hôtel|T2|T3|T1|T4|T5|T6|Duplex/triplex|Loft|Villa|Atelier|studio'
    
    #Avoid local variable error
    global i
    
    #Loop to reach and clean real estate prices data
    for p in prices :
        price.append((p.text.replace("€", "").replace("\r","").replace("\n","").replace('*','').replace(" ", "")))
            
    #Reaching Cities names and zip codes        
    for c in title:
        if c.find('cite'):
            emp.append(c.find('cite').text.replace('\r','').replace('\n','').replace("\t",""))
        else:
            emp.append(bot)

    #Reaching houses type 
    for t in title:
        
        if t.find('span'):
            for i in re.findall(model_title, t.find('span').text):
                type_h.append(i)
            if i not in re.findall(model_title, t.find('span').text):
                type_h.append(bot)
    
    #Reaching number of pieces in a house
    for b in title :

        if b.find('span'):
            
            for i in re.findall('\n[0-9]{1,2}| [0-9]{1,2} p|[0-9]{1,2}p', b.text):
                
                pieces_nb.append(int(i.replace(' p','').replace('\n','').replace("p","")))
                
            if i not in re.findall('\n[0-9]{1,2}| [0-9]{1,2} p|[0-9]{1,2}p', b.text):
                pieces_nb.append(bot)
   
    #Reaching areas of houses            
    for b in title:
        if b.find('span'):
            for i in (re.findall('[0-9]+ m|[0-9]+m', b.text)):
                areas.append(i.replace(' m','').replace('m',''))
                
            if i not in (re.findall('[0-9]+ m|[0-9]+m', b.text)) :
                areas.append(bot) 
        
        

    #Reaching descriptions of houses
    for d in desc:
        description.append(d.text.replace("\r","").replace("\n",""))

In [8]:
#Function to change pages
def get_pages(count=int):
    for pages_nb in range(1,count+1):
        url = f"https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?%20tt=1&at=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50&p={pages_nb}"
        urls.append(url)
    loop_pages(urls,emp,code)
    

#Script where we put urls list to enable all the functions

#x stands for urls, y for emp and z for code
def loop_pages(x,y,z):
    for n in range(len(x)):
        page = requests.get(x[n])
        soup = bs(page.content, 'html.parser')
        parse_pages(x[n], soup)
        pages.append(page)
    zip_code(y)
    dep_(z)
    lists_to_df(type_h,pieces_nb,areas,price,dep,code,emp,description)
        


In [9]:
#Launch program

In [ ]:
get_pages()

In [11]:
#All the code below can be used to check the datas

In [ ]:
len(emp),len(price),len(areas),len(type_h),len(description),len(pieces_nb),len(code),len(dep)

In [ ]:
print(pieces_nb)

In [ ]:
print(code)

In [ ]:
print(emp)

In [ ]:
print(areas)

In [ ]:
print(type_h)

In [ ]:
print(urls)